In [2]:
import pandas as pd
import numpy as np

mush_df=pd.read_csv('mushrooms.csv')
mush_df.isnull().mean()
mush_df_encoding=pd.get_dummies(mush_df)
mush_df_encoding.head()


,class_e,class_p,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [3]:
mush_df_encoding.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Columns: 119 entries, class_e to habitat_w
dtypes: uint8(119)
memory usage: 944.2 KB


In [4]:
# 将特征和类别标签分布赋值给 X 和 y
X_mush = mush_df_encoding.iloc[:,2:]
y_mush = mush_df_encoding.iloc[:,1]

In [4]:
X_mush.head()

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [5]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
pca=PCA(n_components=15,whiten=True,random_state=1)
svc=SVC(kernel='linear',class_weight='balanced')
model=make_pipeline(pca,svc)

# 划分数据集

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_mush,y_mush,random_state=2)
X_train.head()

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
963,0,0,0,0,0,1,1,0,0,0,...,0,0,1,1,0,0,0,0,0,0
1289,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3477,0,0,0,0,0,1,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4692,0,0,0,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
5118,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0


In [7]:
y_train.value_counts()#是否存在样本平衡问题
y_test.value_counts()

0    1083
1     948
Name: class_p, dtype: int64

# 调参

In [8]:
from sklearn.model_selection import GridSearchCV

grid_dic={'svc__C':[1,5,10,50]}
grid=GridSearchCV(model,grid_dic)

%time grid.fit(X_train,y_train)
print(grid.best_params_)

E:\learning\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Wall time: 13.2 s
{'svc__C': 5}


# 预测并计算准确率

In [9]:
model = grid.best_estimator_
model.score(X_test,y_test)

0.9645494830132939

# 使用Kernel

In [12]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
                       scoring='precision')
clf.fit(X_train, y_train)
print(clf.best_params_)
model = grid.best_estimator_
model.score(X_test,y_test)

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}


0.9645494830132939